In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ukraine-war-tweets-dataset-65-days/Russia_invade.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/Russian_border_Ukraine.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/Ukraine_nato.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/Ukraine_troops.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/Russian_troops.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/Ukraine_border.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/StandWithUkraine.csv
/kaggle/input/ukraine-war-tweets-dataset-65-days/Ukraine_war.csv


In [2]:
pip install twython

Note: you may need to restart the kernel to use updated packages.


In [3]:
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import re
import string
import seaborn as sns
 
 
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:

tweets1=pd.read_csv('/kaggle/input/ukraine-war-tweets-dataset-65-days/Russian_border_Ukraine.csv')
tweets2=pd.read_csv('/kaggle/input/ukraine-war-tweets-dataset-65-days/Ukraine_war.csv')
tweets=pd.concat([tweets1, tweets2], ignore_index=True)

In [5]:
tweets.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Searh
0,snscrape.modules.twitter.Tweet,https://twitter.com/donveach/status/1500259722...,2022-03-05 23:59:25+00:00,Is that the same expensive and ineffective bor...,Is that the same expensive and ineffective bor...,1500259722112520194,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,NaN,"{'_type': 'snscrape.modules.twitter.Tweet', 'u...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Russian border Ukraine
1,snscrape.modules.twitter.Tweet,https://twitter.com/deliveryman199/status/1500...,2022-03-05 23:58:30+00:00,@DonWaifu @gabehayes @aaronjmate So the Americ...,@DonWaifu @gabehayes @aaronjmate So the Americ...,1500259491232641025,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,2,...,NaN,NaN,1.500251e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Russian border Ukraine
2,snscrape.modules.twitter.Tweet,https://twitter.com/JDiozzi/status/15002591158...,2022-03-05 23:57:01+00:00,Maybe if one military unit from Ukraine crosse...,Maybe if one military unit from Ukraine crosse...,1500259115863777280,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Russian border Ukraine
3,snscrape.modules.twitter.Tweet,https://twitter.com/Mom2ShihTzu/status/1500258...,2022-03-05 23:55:31+00:00,Volunteers cross Polish border into Ukraine to...,Volunteers cross Polish border into Ukraine to...,1500258740423086083,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Russian border Ukraine
4,snscrape.modules.twitter.Tweet,https://twitter.com/SiliconMatters/status/1500...,2022-03-05 23:54:18+00:00,@MarkHertling BUT... theres no Russian aircraf...,@MarkHertling BUT... theres no Russian aircraf...,1500258434414874625,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,1,4,...,NaN,NaN,1.500223e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Russian border Ukraine


**Exploratory Data Analysis**

In [6]:
print(tweets.shape)

(284664, 29)


In [7]:
import random
random_state=42
# print a random sample of 10 rows from the dataframe
display(tweets.sample(n=10, random_state=random_state))

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Searh
272783,snscrape.modules.twitter.Tweet,https://twitter.com/germanedcantorm/status/148...,2022-01-15 22:43:10+00:00,Putin presents a profound threat to peace in E...,Putin presents a profound threat to peace in E...,1482483527006498816,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ukraine war
279767,snscrape.modules.twitter.Tweet,https://twitter.com/dh_a_heller/status/1480570...,2022-01-10 16:02:12+00:00,"As part of a two-day trip to Ukraine, EU forei...","As part of a two-day trip to Ukraine, EU forei...",1480570678764048389,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,NaN,1.480571e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",NaN,NaN,NaN,NaN,NaN,Ukraine war
86249,snscrape.modules.twitter.Tweet,https://twitter.com/RoodbariIraj/status/149807...,2022-02-27 23:17:35+00:00,@AriaParsa13 تنها بازنده جنگها، مردم بیگناه هس...,@AriaParsa13 تنها بازنده جنگها، مردم بیگناه هس...,1498074867542986760,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,2,...,NaN,NaN,1.498012e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,"['نه_به_جنگ', 'stop_war_in_ukraine']",NaN,Ukraine war
51229,snscrape.modules.twitter.Tweet,https://twitter.com/jodisp2019/status/14823663...,2022-01-15 14:57:32+00:00,"A ""high stakes"" trip the article says. What no...","A ""high stakes"" trip the article says. What no...",1482366344716816387,"{'_type': 'snscrape.modules.twitter.User', 'us...",3,7,24,...,NaN,"{'_type': 'snscrape.modules.twitter.Tweet', 'u...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Russian border Ukraine
169997,snscrape.modules.twitter.Tweet,https://twitter.com/Lee_Chris5/status/14918029...,2022-02-10 15:55:20+00:00,@TulsiGabbard @killaxxxcam The TV stations wer...,@TulsiGabbard @killaxxxcam The TV stations wer...,1491802976360357890,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,4,...,NaN,NaN,1.491713e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Ukraine war
243782,snscrape.modules.twitter.Tweet,https://twitter.com/radicalgraffiti/status/148...,2022-01-24 17:42:40+00:00,"@_____newt i doubt there will be a direct war,...","@_____newt i doubt there will be a direct war,...",1485669393669447683,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,NaN,NaN,1.485654e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Ukraine war
166327,snscrape.modules.twitter.Tweet,https://twitter.com/CPGBML/status/149223755921...,2022-02-11 20:42:13+00:00,Keir #Starmer (proudly) notes “Attlee and his ...,Keir #Starmer (proudly) notes “Attlee and his ...,1492237559212257284,"{'_type': 'snscrape.modules.twitter.User', 'us...",2,1,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Starmer', 'Labour', 'Nato', 'Iraq', 'Hitler'...",NaN,Ukraine war
108355,snscrape.modules.twitter.Tweet,https://twitter.com/MarJesPaul/status/14962678...,2022-02-22 23:37:08+00:00,@WalshFreedom @MalcolmNance The deaths coming ...,@WalshFreedom @MalcolmNance The deaths coming ...,1496267845034033157,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,1,1,...,NaN,NaN,1.496240e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,Ukraine war
10526,snscrape.modules.twitter.Tweet,https://twitter.com/abhijeetg/status/149793529...,2022-02-27 14:02:59+00:00,Breaking News\n\n#Ukrainian and #Russian count...,Breaking News\n\n#Ukrainian and #Russian count...,1497935297975566340,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Ukrainian', 'Russian', 'Belarus', 'Ukrania',...",NaN,Russian border Ukraine
148974,sns

In [8]:
# show the number of missing rows in each column
print(tweets.isna().sum().sort_values(ascending=False))

retweetedTweet      284664
cashtags            283882
place               281222
coordinates         281222
quotedTweet         261553
media               254633
hashtags            228651
tcooutlinks         190927
outlinks            190927
inReplyToUser       155051
inReplyToTweetId    155051
mentionedUsers      137847
_type                    0
sourceLabel              0
sourceUrl                0
url                      0
source                   0
lang                     0
conversationId           0
quoteCount               0
likeCount                0
retweetCount             0
replyCount               0
user                     0
id                       0
renderedContent          0
content                  0
date                     0
Searh                    0
dtype: int64


In [9]:
# print summary info on each column of the dataframe
print(tweets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284664 entries, 0 to 284663
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   _type             284664 non-null  object 
 1   url               284664 non-null  object 
 2   date              284664 non-null  object 
 3   content           284664 non-null  object 
 4   renderedContent   284664 non-null  object 
 5   id                284664 non-null  int64  
 6   user              284664 non-null  object 
 7   replyCount        284664 non-null  int64  
 8   retweetCount      284664 non-null  int64  
 9   likeCount         284664 non-null  int64  
 10  quoteCount        284664 non-null  int64  
 11  conversationId    284664 non-null  int64  
 12  lang              284664 non-null  object 
 13  source            284664 non-null  object 
 14  sourceUrl         284664 non-null  object 
 15  sourceLabel       284664 non-null  object 
 16  outlinks          93

**DATA CLEANING**

In [10]:
tweets.drop_duplicates(inplace=True)  # remove duplicates

In [11]:
# slicing the date , and removing the time portion
tweets['date'] = tweets.date.str.slice(0, 10)

In [12]:
# checking all the unique dates in the dataset
print(tweets['date'].unique())  
 
# checking how many unique language
# tweets are present in the dataset
print(tweets["lang"].unique())

['2022-03-05' '2022-03-04' '2022-03-03' '2022-03-02' '2022-03-01'
 '2022-02-28' '2022-02-27' '2022-02-26' '2022-02-25' '2022-02-24'
 '2022-02-23' '2022-02-22' '2022-02-21' '2022-02-20' '2022-02-19'
 '2022-02-18' '2022-02-17' '2022-02-16' '2022-02-15' '2022-02-14'
 '2022-02-13' '2022-02-12' '2022-02-11' '2022-02-10' '2022-02-09'
 '2022-02-08' '2022-02-07' '2022-02-06' '2022-02-05' '2022-02-04'
 '2022-02-03' '2022-02-02' '2022-02-01' '2022-01-31' '2022-01-30'
 '2022-01-29' '2022-01-28' '2022-01-27' '2022-01-26' '2022-01-25'
 '2022-01-24' '2022-01-23' '2022-01-22' '2022-01-21' '2022-01-20'
 '2022-01-19' '2022-01-18' '2022-01-17' '2022-01-16' '2022-01-15'
 '2022-01-14' '2022-01-13' '2022-01-12' '2022-01-11' '2022-01-10'
 '2022-01-09' '2022-01-08' '2022-01-07' '2022-01-06' '2022-01-05'
 '2022-01-04' '2022-01-03' '2022-01-02' '2022-01-01' '2021-12-31']
['en' 'es' 'und' 'hi' 'in' 'de' 'ja' 'pl' 'et' 'zh' 'ro' 'nl' 'tr' 'pa'
 'da' 'pt' 'tl' 'eu' 'fr' 'no' 'cs' 'ru' 'fi' 'it' 'sv' 'ca' 'kn' 'sl

In [13]:
# before removing the non-english tweets
print(tweets.shape)

# removing all the tweets expect the
# non-english tweets
tweets = tweets[tweets['lang'] == 'en']
print("After removing non-english Tweets")

# only the number of english tweets
print(tweets.shape) 


(284664, 29)
After removing non-english Tweets
(272482, 29)


In [14]:
import random

def print_random_tweet() -> None:
    random_tweet = tweets["content"].sample(n=1, random_state=random_state).values
    print(random_tweet[0])

print_random_tweet()


Given what is happening in Ukraine Russia now we know what trump did by withholding aide to be so criminal and evil given the conflict hazard in that region; he wasn’t even convicted of extortion during a war conflict - this country is full of idiots


In [15]:
# Removing RT, Punctuation etc
def remove_rt(x): return re.sub('RT @\w+: ', " ", x)
 
def rt(x): return re.sub(
    "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x)
tweets.loc[:, "content"] = tweets["content"].map(remove_rt).map(rt)
print_random_tweet()

Given what is happening in Ukraine Russia now we know what trump did by withholding aide to be so criminal and evil given the conflict hazard in that region  he wasn t even convicted of extortion during a war conflict   this country is full of idiots


In [16]:
tweets.loc[:, "content"] = tweets["content"].str.lower()
print_random_tweet()

given what is happening in ukraine russia now we know what trump did by withholding aide to be so criminal and evil given the conflict hazard in that region  he wasn t even convicted of extortion during a war conflict   this country is full of idiots


In [17]:
def remove_numerical(text):
    return ''.join([word for word in text if not word.isdigit()])

tweets.loc[:, "content"] = tweets["content"].str.lower().apply(remove_numerical)
print_random_tweet()

given what is happening in ukraine russia now we know what trump did by withholding aide to be so criminal and evil given the conflict hazard in that region  he wasn t even convicted of extortion during a war conflict   this country is full of idiots


In [18]:
import pandas as pd

def remove_non_sensical(text):
    return text.replace('\n', '').replace('\r', '').replace('\t', '')

# Assuming tweets is your DataFrame
tweets.loc[:, "content"] = tweets["content"].str.lower().apply(remove_non_sensical)


In [19]:
# Function to tokenize text using split()
def tokenize_text(text):
    tokens = text.split()
    return tokens

# Apply the tokenize_text function to your DataFrame column
tweets['tokenized_review'] = tweets['content'].apply(tokenize_text)

# Show the DataFrame with tokenized reviews
print(tweets['tokenized_review'])


0         [is, that, the, same, expensive, and, ineffect...
1         [so, the, americans, are, dressing, up, as, ru...
2         [maybe, if, one, military, unit, from, ukraine...
3         [volunteers, cross, polish, border, into, ukra...
4         [but, theres, no, russian, aircraft, flying, n...
                                ...                        
284659    [rusia, wants, to, take, back, ukraine, they, ...
284660    [a, positive, for, russia, is, that, according...
284661    [resists, further, invades, because, already, ...
284662    [i, fully, support, ukraine, s, sovereignty, b...
284663    [biden, warns, putin, what, does, acting, deci...
Name: tokenized_review, Length: 272482, dtype: object


In [20]:
corpus = tweets[['content']].copy()

# Display the corpus
print("Corpus:")
print(corpus)

Corpus:
                                                  content
0       is that the same expensive and ineffective bor...
1             so the americans are dressing up as russ...
2       maybe if one military unit from ukraine crosse...
3       volunteers cross polish border into ukraine to...
4         but    theres no russian aircraft flying nea...
...                                                   ...
284659  rusia wants to take back ukraine  they are our...
284660  a positive for russia is that  according to us...
284661      resists  further invades  because already ...
284662      i fully support ukraine s sovereignty  but...
284663  biden warns putin   what does acting decisivel...

[272482 rows x 1 columns]


**DOCUMENT TERM MATRIX**

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming df1 is your DataFrame with more than 100 entries
# Sample data (assuming df1 is your DataFrame)
df1_sample = tweets.head(1000)  # Selecting the first 100 entries

# Step 1: Create a CountVectorizer object
cv = CountVectorizer(stop_words='english')

# Step 2: Fit and transform the data to create the document-term matrix
data_cv = cv.fit_transform(df1_sample['content'])

# Step 3: Convert the document-term matrix to a DataFrame
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())

# Display the document-term matrix
print(data_dtm)


     aa  abandoned  abhinandan  ability  able  abm  abortion  absolute  \
0     0          0           0        0     0    0         0         0   
1     0          0           0        0     0    0         0         0   
2     0          0           0        0     0    0         0         0   
3     0          0           0        0     0    0         0         0   
4     0          0           0        0     0    0         0         0   
..   ..        ...         ...      ...   ...  ...       ...       ...   
995   0          0           0        0     0    0         0         0   
996   0          0           0        0     0    0         0         0   
997   0          0           0        0     0    0         0         0   
998   0          0           0        0     0    0         0         0   
999   0          0           0        0     0    0         0         0   

     absolutely  absorbed  ...  zelenksy  zelensky  zelenskyy  zhytomyr  \
0             0         0  ...      

In [22]:
# Define the function to clean text for round 3
def clean_text_round3(text):
    text = re.sub('–', '', text)
    text = re.sub('\xa0', '', text)
    text = re.sub('[♪]', '', text)
    text = re.sub('\s+', ' ', text.strip())
    return text

# Apply the clean_text_round3 function to the 'content' column of your DataFrame
tweets['clean_content'] = tweets['content'].apply(clean_text_round3)


In [23]:
tweets['clean_content'].head()

0    is that the same expensive and ineffective bor...
1    so the americans are dressing up as russians c...
2    maybe if one military unit from ukraine crosse...
3    volunteers cross polish border into ukraine to...
4    but theres no russian aircraft flying near lvi...
Name: clean_content, dtype: object

In [24]:
# Create a CountVectorizer object with specified parameters
cv = CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=0.2, max_df=0.8)

# Fit and transform the data to create the document-term matrix
data_cv = cv.fit_transform(corpus['content'])

# Convert the document-term matrix to a DataFrame
data_dtm2 = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())

# Display the document-term matrix
print(data_dtm2)

        border  russia  russian  war
0            1       0        1    0
1            1       0        3    0
2            1       0        1    0
3            1       0        1    0
4            1       1        1    0
...        ...     ...      ...  ...
272477       0       0        0    1
272478       0       1        0    1
272479       0       1        0    1
272480       0       2        0    1
272481       0       1        0    1

[272482 rows x 4 columns]


In [25]:
print(data_dtm.shape) 
print(data_dtm2.shape)

(1000, 4502)
(272482, 4)


In [26]:
import pickle

# Assuming `df` is your DataFrame
# Save the DataFrame to a pickle file
with open('dtm.pkl', 'wb') as f:
    pickle.dump(data_dtm2, f)

print("DataFrame saved to dtm.pkl")


DataFrame saved to dtm.pkl


In [27]:
import pickle

# Assuming `df` is your DataFrame
# Save the DataFrame to a pickle file
with open('dataframe.pkl', 'wb') as f:
    pickle.dump(tweets, f)

print("DataFrame saved to dataframe.pkl")


DataFrame saved to dataframe.pkl
